In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from itertools import islice
from datetime import datetime
from pathlib import Path

import joblib
import pandas as pd
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, average_precision_score

from plotting import (
    plot_confusion_matrix,
    plot_correct_prediction_counts,
    plot_sepsis_prediction_evolution, plot_combined_patient_data,
)

#### Load preprocessed splits

In [ ]:
DATA_PATH = Path("../data")
SPLITS_PATH = DATA_PATH / "splits"

In [ ]:
TARGET_COL = "SepsisLabel"
ID_COL = "Patient_ID"
TIME_COL = "ICULOS"

In [ ]:
train_df = pd.read_parquet(SPLITS_PATH / "train.parquet", engine="pyarrow")
val_df   = pd.read_parquet(SPLITS_PATH / "val.parquet", engine="pyarrow")
test_df  = pd.read_parquet(SPLITS_PATH / "test.parquet", engine="pyarrow")

In [ ]:
print(train_df.shape)
print(list(train_df.columns))

In [ ]:
artifacts = joblib.load(DATA_PATH / "preprocess_artifacts.pkl")

In [ ]:
feature_cols = [c for c in train_df.columns if c not in [TARGET_COL, ID_COL, TIME_COL]]

In [ ]:
X_train = train_df[feature_cols]
y_train = train_df[TARGET_COL]

X_val = val_df[feature_cols]
y_val = val_df[TARGET_COL]

X_test = test_df.copy()
X_test_feat = test_df[feature_cols]
y_test = test_df[TARGET_COL]

#### Categorical Features

In [ ]:
cat_feature_names = ["Gender"]

train_pool = Pool(X_train, y_train, cat_features=cat_feature_names or None)
val_pool = Pool(X_val, y_val, cat_features=cat_feature_names or None)

#### Calculate Positive Class Weight

In [ ]:
pos_weight = (y_train == 0).sum() / max((y_train == 1).sum(), 1)
class_weights = [1.0, pos_weight]

#### Model Params

In [ ]:
# Best params after Param Grid
best_params = {
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "iterations": 10000,
    "od_type": "Iter",
    "od_wait": 150,
    "random_strength": 10,
    "learning_rate": 0.01,
    "depth": 6,
    "l2_leaf_reg": 3,
    "bagging_temperature": 1,
    "border_count": 128,
    "class_weights": class_weights,
    "random_state": 42,
    "task_type": "GPU",
    "metric_period": 1,
    "use_best_model": True,
}


#### Train

In [ ]:
final_model = CatBoostClassifier(
    train_dir="../catboost/",
    **best_params
)
final_model.fit(train_pool, eval_set=val_pool, verbose=True)

In [ ]:
now_str = datetime.now().strftime("%Y-%match-%d_%H-%M-%S")
final_model.save_model(f"../models/{now_str}-model.cbm")
print("Saved best model.")

#### Test and Metrics

In [ ]:
threshold = 0.5

In [ ]:
test_probs = final_model.predict_proba(X_test_feat)[:, 1]
test_probs_s = pd.Series(test_probs, index=X_test.index, name="PredProb")
test_pred = (test_probs_s >= threshold).astype(int)

X_test_with_preds = X_test.copy()
X_test_with_preds["PredProb"] = test_probs_s

In [ ]:
print(classification_report(y_test, test_pred))
print(roc_auc_score(y_test, test_pred))
print(average_precision_score(y_test, test_pred))

In [ ]:
plot_confusion_matrix(confusion_matrix(y_test, test_pred))

In [ ]:
positive_ids = (
    X_test_with_preds.groupby(ID_COL)[TARGET_COL]
          .max()
          .loc[lambda s: s == 1]
          .index
)

In [ ]:
X_test_positive = X_test_with_preds[X_test_with_preds[ID_COL].isin(positive_ids)].copy()

In [ ]:
X_test_positive["PredLabel"] = (
    (X_test_positive["PredProb"].to_numpy() > threshold).astype(int)
)

In [ ]:
H: int = 0  # Nth patient hour, where SepsisLabel==1; {H ∈ ℤ : 0 <= H < L_min}, where L_min is the minimum patient sequence length

In [ ]:
nth_sepsis_rows = (
    X_test_positive[X_test_positive[TARGET_COL] == 1]
    .sort_values([ID_COL, TIME_COL])
    .groupby(ID_COL, as_index=False)
    .nth(H)
    [[ID_COL, TIME_COL, TARGET_COL, "PredProb", "PredLabel"]]
)

In [ ]:
patients_with_wrong_preds = nth_sepsis_rows.loc[nth_sepsis_rows["PredLabel"] == 0]["Patient_ID"]

In [ ]:
wrong_preds_df = X_test_positive[
    X_test_positive["Patient_ID"].isin(patients_with_wrong_preds)
]

In [ ]:
plot_correct_prediction_counts(nth_sepsis_rows["PredLabel"].to_numpy(), H)

In [ ]:
N = 10  # Amount of patients with incorrect predictions to display
for patient_id, patient_df in islice(wrong_preds_df.groupby(ID_COL), H):
    patient_df = patient_df.sort_values(TIME_COL)
    plot_sepsis_prediction_evolution(patient_df, patient_id=patient_id, threshold=threshold)
    plot_combined_patient_data(patient_df)

#### Export predictions to calculate Physionet 2019 Challenge score
**Note:** This score is not 100% accurate because the official scoring test dataset is not publicly available.

In [ ]:
PREDS_DIR = Path("../preds/")

LABEL_DIR = PREDS_DIR / "eval_labels/"
PRED_DIR = PREDS_DIR / "eval_preds/"

LABEL_DIR.mkdir(parents=True, exist_ok=True)
PRED_DIR.mkdir(parents=True, exist_ok=True)

for pid, pdf in X_test_with_preds.groupby("Patient_ID"):
    pdf = pdf.sort_values("ICULOS")

    label_df = pdf[["SepsisLabel"]].copy()
    label_path = LABEL_DIR / f"{pid}.psv"
    label_df.to_csv(label_path, sep='|', index=False)

    pred_df = pd.DataFrame({
        "PredictedProbability": pdf["PredProb"].values,
        "PredictedLabel": pdf["PredLabel"].values
    })
    pred_path = PRED_DIR / f"{pid}.psv"
    pred_df.to_csv(pred_path, sep='|', index=False)